In [1]:
import os
import sys
from time import gmtime, strftime

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import torch
import poutyne
from poutyne import Model,Experiment

In [3]:
from data.custom_data import filepath_dataframe
from data.selection import Selection
from data.transformation import Transform_CnnLstmS,Transform_CnnS
from data.torchData import DataLoadings,DataLoading

from data.custom_data import nucPaired_fpDataframe
from data.torchData import PairDataLoading,DataLoading
from training.contrastive_pretraining import Contrastive_PreTraining
from training.finetuning import FineTuneCNN

import models

In [5]:
#####################################################################################################################

# random seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# gpu setting
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(DEVICE)
device = DEVICE

## data directory
data_dir  = 'E:\\external_data\\opera_csi\\Session_2\\experiment_data\\experiment_data\\exp_7_amp_spec_only\\npy_format'
readtype = 'npy'
splitchar = '\\'
fpath = '.'

# data selection
dataselection_name = 'EXP7-NUC1-Room1-Amp-RandomSplit-ResReduced'

data_selection = Selection(split='random',test_sub=0.2,val_sub=0.1,
                           nuc='NUC1',room=1,sample_per_class=None)

# data loading
transform = Transform_CnnS()
batch_size = 64
num_workers = 0

# training
optimizer_builder = torch.optim.SGD
lr = 0.001
pretrain_epochs = 1
finetune_epochs = 1

# model
builder = models.cnn.create_alexnet
network_name = 'AlexNet'

# Experiment Name
comment = ''
exp_name = f'{network_name}_NTXent_{dataselection_name}_Comment-{comment}'

# auto
record_dir = os.path.join(fpath,'record')
model_dir = os.path.join(fpath,'saved_model')
model_fname = os.path.join(model_dir,f'Encoder___{exp_name}')
print('Experiment Name: ',exp_name)
print('Cuda Availability: ',torch.cuda.is_available())

Experiment Name:  AlexNet_Supervised_EXP7-NUC1-Room1-Amp-RandomSplit-ResReduced_Comment-TestConLn
Cuda Availability:  True


In [6]:
# -----------------------------------Main-------------------------------------------

# if __name__ == '__main__':

# data preparation
df = filepath_dataframe(data_dir,splitchar)
df_train,df_val,df_test = data_selection(df)

In [9]:
##### FINE-TUNING #####

# data loading
data_loading = DataLoading(transform=transform,batch_size=batch_size,readtype=readtype,
                           num_workers=num_workers,drop_last=True)
test_loading = DataLoading(transform=transform,batch_size=len(df_test),readtype=readtype,
                           num_workers=num_workers,drop_last=True)

train_loader = data_loading(df_train)
val_loader   = data_loading(df_val)
test_loader  = test_loading(df_test)

# load and create model
model = FineTuneCNN(model_path=model_fname,
                    encoder_builder=builder,
                    n_classes=df.activity.nunique())

In [10]:
# train with poutyne
mdl = Model(model,'adam','cross_entropy',
            batch_metrics=['accuracy'],
            epoch_metrics=[poutyne.F1('micro'),poutyne.F1('macro')]).to(device)
history = mdl.fit_generator(train_generator=train_loader,valid_generator=test_loader,epochs=finetune_epochs)

Epoch: 1/20 Step: 20/20 100.00% |█████████████████████████|36.57s loss: 1106.835609 acc: 29.921875 fscore_micro: 0.299219 fscore_macro: 0.227387 val_loss: 349.071716 val_acc: 30.864197 val_fscore_micro: 0.308642 val_fscore_macro: 0.204785
Epoch: 2/20 Step: 20/20 100.00% |█████████████████████████|36.18s loss: 177.697598 acc: 34.609375 fscore_micro: 0.346094 fscore_macro: 0.271254 val_loss: 32.411316 val_acc: 25.308641 val_fscore_micro: 0.253086 val_fscore_macro: 0.240315
Epoch: 3/20 Step: 20/20 100.00% |█████████████████████████|36.44s loss: 34.033852 acc: 33.359375 fscore_micro: 0.333594 fscore_macro: 0.234944 val_loss: 22.540155 val_acc: 35.802467 val_fscore_micro: 0.358025 val_fscore_macro: 0.270895
Epoch: 4/20 Step: 20/20 100.00% |█████████████████████████|36.41s loss: 21.882629 acc: 33.046875 fscore_micro: 0.330469 fscore_macro: 0.261862 val_loss: 11.339959 val_acc: 27.160492 val_fscore_micro: 0.271605 val_fscore_macro: 0.182859
Epoch: 5/20 Step: 20/20 100.00% |███████████████████